In [ ]:
%matplotlib inline

# 機械学習ハンズオン

機械学習で何か作る際の一連の流れを通して、身近な問題解決手段として選べるようにする。
今回は手軽なkerasを利用してCNN(画像分類)を行います。

1. データ収集(データベース、データセット、スクレイピング)
2. 加工(リサイズ、水増し、画像処理)
3. トレーニング(モデル設計、テスト)
4. アプリケーションに組み込み

In [ ]:
search_texts = ['犬', '猫']
datasets_path = 'volume/images'
model_path = 'volume/models/model.h5'
image_size = 28

## データ収集

まずは分類テーマを決めてその画像を集めます。
自社のデータが使えたり、データセットがあれば良いですが、ない場合は自分で収集する必要があります。
今回はスクレピングして画像を集めます。

スクレピングするライブラリはBeautifulSoupを利用します。
下記の例はgithub月間トレンド1位のリポジトリを調べるscriptです。
BeautifulSoupでhtmlを解析して、selectでセレクタ指定して要素を取得できます。(:nth-childとかは効かない?)

In [ ]:
import os, shutil
import requests
from bs4 import BeautifulSoup

In [ ]:
html = requests.get('https://github.com/trending?since=monthly').text
soup = BeautifulSoup(html, 'html.parser')
element = soup.select('ol.repo-list > li h3 a')[0]
print(element.text)
print('https://github.com{}'.format(element.attrs['href']))

下記のgoogleで画像検索して保存する関数を使って1ページ20枚、10ページ分の画像を収集。
search_textsごとにvolume/images/{index}に保存します。

In [ ]:
def image_scraping(search_texts=[], save_path='.', page=1):
    for search_index, search_text in enumerate(search_texts):
        
        # googleで画像検索
        url = 'https://www.google.co.jp/search?q={}&tbm=isch'.format(search_text)
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        
        for page_index in range(page):
            
            # 画像を拾う
            image_elements = soup.select('#ires a > img')
            for element_index, image_element in enumerate(image_elements):
                
                # src属性から画像をダウンロード
                image_url = image_element.attrs['src']
                response = requests.get(image_url, stream=True)
                dest_path = '{}/{}'.format(save_path, search_index)
                if not os.path.isdir(dest_path):
                    os.makedirs(dest_path)
                file_path = '{}/{}.png'.format(dest_path, (page_index * len(image_elements)) + element_index)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        response.raw.decode_content = True
                        shutil.copyfileobj(response.raw, file)
                        print('download: {}'.format(file_path))
                        
            # 次のページ
            page_links = soup.select('#nav tr td a')
            next_url = 'https://www.google.co.jp{}'.format(page_links[-1].attrs['href'])
            html = requests.get(next_url).text
            soup = BeautifulSoup(html, 'html.parser')

In [ ]:
image_scraping(search_texts=search_texts, save_path=datasets_path, page=10)

## 加工

集めたデータは加工する必要があります。
学習する為にはその画像のサイズを統一する必要があり(必要ないモデルもあります)、今回は28x28のカラー画像で統一します。

集めた画像のディレクトリをラベルとして利用します。

In [ ]:
import sys, os, glob
import cv2
import numpy as np

In [ ]:
def get_dirs(src_path):
    listdir = os.listdir(src_path)
    return [path for path in listdir if os.path.isdir(os.path.join(src_path, path))]

In [ ]:
get_dirs(datasets_path)

下記の関数を使って保存されている画像に対して28x28のカラー画像に変換します。

In [ ]:
def image_resize(src_path, size=28):
    dirs = get_dirs(src_path)
    for dir_path in dirs:
        for image_path in glob.glob('{}/{}/**/*.*'.format(src_path, dir_path), recursive=True):
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = cv2.resize(image, (size, size))
            file_name = os.path.basename(image_path)
            save_path = '{}/{}/{}'.format(src_path, dir_path, file_name)
            cv2.imwrite(save_path, image)
            print('resize: {}'.format(save_path))

In [ ]:
image_resize(datasets_path, size=image_size)

よりよい学習の為にはまず一番にデータをたくさん集める必要があります。
また、分類ごとに枚数をなるべく統一した方がいいです。
データを集める手法に水増しがあります。

これもkerasでImageDataGeneratorという関数が用意されているのでそれを利用します。

犬や猫の画像は正面から撮られている事が多いのでx軸回転はなしにしています。
携帯など上下関係なく判断したい場合はx軸回転を有効にするなどデータによってパラメータを調整してください。

In [ ]:
import os, sys, glob, imghdr, time
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
def image_generate(src_path, rate=10):
    datagen = ImageDataGenerator(
        # rotation_range=40, #z軸回転
        width_shift_range=0.05, #水平
        height_shift_range=0.05, #上下
        # shear_range=0.1, #斜め引き伸ばし
        zoom_range=0.1, #ズーム
        # horizontal_flip=True, #x軸回転
        vertical_flip=True, #y軸回転
        fill_mode='nearest'
    )
    for image_path in glob.glob('{}/**/*.*'.format(src_path), recursive=True):
        img = load_img(image_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)
        generator = datagen.flow(
            x,
            batch_size=1,
            save_to_dir=src_path,
            save_prefix='gen',
            save_format='png'
        )
        for _ in range(rate):
            generator.next()
        print('generate: {}'.format(image_path))

In [ ]:
for dir_name in get_dirs(datasets_path):
    target_path = '{}/{}'.format(datasets_path, dir_name)
    image_generate(target_path, 2)

データの加工手法はたくさんあります。
輪郭が重要な意味をもつデータに対してはエッジ検出、マイコンに組み込んで判断速度が重要な問題(自動ブレーキなど)には0か1のバイナリ化するなど。

## トレーニング

tensorflow 1.4からkerasが正式に組み込まれたのでtensroflow内蔵のkerasを利用します。
kerasはtensorflow(とTheano)のラッパーなのでどちらにせよtensorflowが必要。

In [ ]:
import os, glob, imghdr
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.python.keras.optimizers import Adadelta
from tensorflow.python.keras.losses import categorical_crossentropy
from tensorflow.python.keras.utils import plot_model, to_categorical
from tensorflow.python.keras import backend as K
from tensorflow.contrib.learn.python.learn.estimators._sklearn import train_test_split

画像をcv2で読み込んでarray likeなnumpyオブジェクトに慣れましょう。
実態は違いますがarrayっぽく動作し、行列演算の為に最適化されてます。

In [ ]:
image = cv2.imread('volume/images/0/0.png')
# 正規化 約分のようなもの 0~1などに正規化する
normalized_image = image.astype(np.float32) / 255.0
print(normalized_image)
print(normalized_image.shape)
# モデルによってはデータをベクトル化して1次元配列にすることもある
vectorize_image = normalized_image.flatten()
print(vectorize_image)
print(vectorize_image.shape)

In [ ]:
def get_normalized_image(image_path, image_size=56):
    """ 画像を正規化する
    @param
        images_path         画像パス
        image_size          画像の1辺のpixel数
    @return
        normalized_image    正規化された画像
    """
    image = cv2.imread(image_path)
    image = cv2.resize(image, (image_size, image_size))
    normalized_image = image.astype(np.float32) / 255.0
    return normalized_image

画像データは28x28x3です。
答えラベル(分類)データはone hot表現で用意します。

```
分類0の場合
y = [1,0,0,0,0,0,0]

分類5の場合
y = [0,0,0,0,0,1,0]
```

train_test_splitはscikit-learn(ニューラルネット以外の機械学習ライブラリ)の関数ですが、トレーニングデータとテストデータを良い感じに分けてくれるので利用します。まだcontribですが、tensroflow内蔵です。

In [ ]:
def get_datasets(datasets_path, test_size=0.1, image_size=56):
    """ 学習用データセットを返す
    @param
        datasets_path       分類名のフォルダに画像が格納されたデータセットのpath
        test_size           テストに使用する画像の割合
        image_size          画像サイズ
    @return
        datasets            tuple(train_images, test_images, train_labels, test_labels)
    """

    labels = get_dirs(datasets_path)

    x = []
    y = []
    for label in labels:
        for image_path in glob.glob('{}/{}/**/*.*'.format(datasets_path, label), recursive=True):
            # 画像形式でなければスキップ
            if imghdr.what(image_path) is None:
                continue

            # 正規化したデータをxへ
            normalized_image = get_normalized_image(image_path, image_size=image_size)
            x.append(normalized_image)

            # one hot表現答えラベルをyへ
            one_hot = np.zeros(len(labels))
            one_hot.put(labels.index(label), 1)
            y.append(one_hot)

    datasets = train_test_split(
        np.array(x),
        np.array(y),
        test_size=test_size
    )
    return datasets

各データの形状を見てみます。

In [ ]:
datasets = get_datasets(datasets_path, test_size=0.1, image_size=image_size)
train_images, test_images, train_labels, test_labels = datasets
print(train_images.shape)
print(test_images.shape)
print(train_labels.shape)
print(test_labels.shape)

1番重要なモデル作成です。
output_shapeで都度データがどういう形状になっていくか確認できます。

- CNNについてhttps://i.stack.imgur.com/jNKSJ.png
- 畳み込み 指定サイズごとに切り出してスライド、そのマッチングを元に増やす
- プーリング 強い特徴のみ残して減らす
- ドロップアウト ニューラルネットワークの線を学習ごとにランダムに切る(過学習抑制) ランダムにできる複数のモデルで学習したような平均化する効果があり、極端にフィットしてしまう事を防ぐ

In [ ]:
def create_model(num_classes, image_size=56, channels=3):
    model = Sequential()

    input_shape = (image_size, image_size, channels)

    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
    print(model.input_shape)
    print(model.output_shape)

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    print(model.output_shape)

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    print(model.output_shape)

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    print(model.output_shape)

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
    print(model.output_shape)

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    print(model.output_shape)

    model.add(Flatten())
    print(model.output_shape)

    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    print(model.output_shape)

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    print(model.output_shape)

    model.add(Dense(num_classes, activation='softmax'))
    print(model.output_shape)

    return model

モデルのインスタンスを作って詳細をconsoleで確認します。

In [ ]:
labels = get_dirs(datasets_path)
num_classes = len(labels)
channels = 3
model = create_model(num_classes, image_size=image_size, channels=channels)

In [ ]:
model.summary()

モデルの詳細を画像に保存できますが tensroflow 1.4.0だと動きませんでした。。

In [ ]:
# # create model image (tensorflow 1.4.0だと動かない)
# plot_model(model, to_file='model.png')

### Q. なぜ一気に(None, 28, 28, 3)を(None, 2)にしないのか

[参考](https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/ch02/xor_gate.py)

![image](http://hokuts.com/wp-content/uploads/2015/11/perceptron.png)

A. XORを作る為

上の画像で例えるなら

- 丸をノードという
- 左側のノードはinputが(28, 28, 3)なので28x28x3=2352個できる
- 右側のノードは最終的に(2)にしたいので2個できる
- 線はデータ(0~1のRGB値)の流れ
- 左側のノードから2352個のデータが右側のノードに渡され、下記のような関数を実行する
- 重みやバイアスの値が変わる事によりgate関数はANDやNANDやOR回路の性質に変化する

```
def gate(*args):
    x = np.array(args)
    w = np.array([0.5, 0.5, ...]) # len(args)個の重み 学習によって変わっていく値
    b = -0.7 # 各ノードに1個存在するバイアス
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1
```

- 例えばこういう関数に変化する

```
def AND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.7
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

def NAND(x1, x2):
    x = np.array([x1, x2])
    w = np.array([-0.5, -0.5])
    b = 0.7
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1

def OR(x1, x2):
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.2
    tmp = np.sum(w*x) + b
    if tmp <= 0:
        return 0
    else:
        return 1
```

- しかしXOR回路が作れない問題がある。
- XOR回路はNAND回路とOR回路の値を受け取って自身がAND回路になると表現できるようになる。

```
def XOR(x1, x2):
    s1 = NAND(x1, x2)
    s2 = OR(x1, x2)
    y = AND(s1, s2)
    return y
```

- 層が少ないと(1層だと確実に)ANDとNANDとOR回路しか作れない

### Q. なぜXOR回路が必要なのか

A. [ここ](http://hokuts.com/2015/12/04/ml3-mlp/)を参考に答えます(ほぼまんまです)



##### ORが役立つパターン

データ分布イメージ

- x_1が男性=0 女性=1
- x_2が辛党=0 甘党=1
- 今週ケーキ屋に行ったかのデータを元に学習
- 性別と甘/辛党のデータを元に来週ケーキ屋に行くかを予測
- お菓子産業で成り立っている国で収集したデータ

|x_1|x_2|x_1 OR x_2|
|---|---|---|
|1|1|1|
|1|0|1|
|0|1|1|
|0|0|0|

![image](http://hokuts.com/wp-content/uploads/2015/12/graph_or.png)

##### ANDが役立つパターン

データ分布イメージ

- x_1が男性=0 女性=1
- x_2が辛党=0 甘党=1
- 今週ケーキ屋に行ったかのデータを元に学習
- 性別と甘/辛党のデータを元に来週ケーキ屋に行くかを予測
- 砂糖が高い国で収集したデータの分布

|x_1|x_2|x_1 OR x_2|
|---|---|---|
|1|1|1|
|1|0|0|
|0|1|0|
|0|0|0|

![image](http://hokuts.com/wp-content/uploads/2015/12/graph_and.png)

##### NANDが役立つパターン

データ分布イメージ

- x_1が男性=0 女性=1
- x_2が辛党=0 甘党=1
- 今週居酒屋に行ったかのデータを元に学習
- 性別と甘/辛党のデータを元に来週居酒屋に行くかを予測

|x_1|x_2|x_1 OR x_2|
|---|---|---|
|1|1|0|
|1|0|1|
|0|1|1|
|0|0|1|

![image](http://hokuts.com/wp-content/uploads/2015/12/graph_nand.png)

##### XORが役立つパターン

データ分布イメージ (例に無理が出てきたw)

- x_1が男性=0 女性=1
- x_2が辛党=0 甘党=1
- 今週飲食店に行ったかのデータを元に学習
- 性別と甘/辛党のデータを元に来週飲食店に行くかを予測
- 辛党男性は居酒屋に、甘党女性はケーキ屋に、それ以外は帰宅する傾向のある国で収集したデータの分布

|x_1|x_2|x_1 OR x_2|
|---|---|---|
|1|1|0|
|1|0|1|
|0|1|1|
|0|0|0|

![image](http://hokuts.com/wp-content/uploads/2015/12/graph_xor1.png)

- ANDとNANDとOR回路のみ(単純パーセプトロン)だと決定境界を線形でしか作れない
- XORに分布したデータに対して単純パーセプトロンで解決できない問題があるから

下記関数を利用してトレーニングを開始します。
batch_size枚を一度に処理し、epochs週トレーニングします。

- データにもよりますが、10分程度で2分類80%くらい
- 2度目も同じmodel使って更新するので何度実行してもOK
- 2度目実行するとtrain_test_splitで毎回ランダムにテストデータが入れ替わるので厳密にはTest accuracyは嘘になる

In [ ]:
def train(datasets_path, model_path, image_size=56, channels=3, test_size=0.1, batch_size=1000, epochs=10):

    datasets = get_datasets(datasets_path, test_size=test_size, image_size=image_size)
    labels = get_dirs(datasets_path)
    num_classes = len(labels)
    train_images, test_images, train_labels, test_labels = datasets

    model = load_model(model_path) if os.path.isfile(model_path) else create_model(num_classes, image_size=image_size, channels=channels)

    model.compile(
        optimizer=Adadelta(),
        loss=categorical_crossentropy,
        metrics=['accuracy']
    )

    print(train_images.shape)

    model.fit(
        train_images,
        train_labels,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(test_images, test_labels)
    )

    model.save(model_path)

    score = model.evaluate(test_images, test_labels, verbose=0)
    print('Test score   : {:>.4f}'.format(score[0]))
    print('Test accuracy: {:>.4f}'.format(score[1]))

    K.clear_session()

In [ ]:
train(
    datasets_path,
    model_path,
    image_size=image_size,
    channels=3,
    test_size=0.1,
    batch_size=2000,
    epochs=200
)

トレーニングした結果を実際に試してみましょう

In [ ]:
image = get_normalized_image('volume/images/0/0.png', image_size=image_size)
model = load_model(model_path)
results = model.predict(np.array([image]), verbose=1)

labels = search_texts
# labels = ['dog', 'cat']

predictions = dict(zip(labels, results[0]))
for label, score in predictions.items():
    print(label, score)

## アプリケーションに組み込み

webに組み込んでみます。
すごく簡単なbottleというpythonのwebフレームワークを使います。

In [ ]:
from json import dumps
from bottle import Bottle, static_file, url, request, response

In [ ]:
app = Bottle()

In [ ]:
@app.get('/:path#.+#')
def get_public(path):
    return static_file(path, root='public')

In [ ]:
@app.get('/')
def get_index():
    return static_file('index.html', root='public')

In [ ]:
# POSTで画像binaryを渡して分類を判定する。ContentType:multipart/form-data
@app.post('/predict/keras')
def post_predict_keras():

    binary = request.files.get('image').file.read()

    # convert binary to image (height any x width any x color channel any)
    image = cv2.imdecode(np.fromstring(binary, np.uint8), cv2.IMREAD_COLOR)

    # convert image size (28 x 28 x 3)
    image = cv2.resize(image, (28, 28))

    model = load_model(model_path)

    predicts = model.predict(np.array([image]))
    predicts = predicts.tolist()
    results = []
    for index, percentage in enumerate(predicts[0]):
        results.append({
            'label': str(index),
            'score': str(percentage)
        })
    results = sorted(results, key=lambda result: result['score'])

    response.content_type = 'application/json'
    return dumps(results)

In [ ]:
@app.get('/test/keras')
def get_test_keras():
    return '''
<form action="/predict/keras" method="post" enctype="multipart/form-data">
    <input type="submit">
    <input type="file" name="image">
</form>
'''

In [ ]:
# 停止ボタンで停止
app.run(host='0.0.0.0', port=8088)

In [ ]:
# http://localhost:8088/test/keras
# http://localhost:8088